In [1]:
import sys
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python/")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/")
sys.path.append("/home/rossidaniel/Dropbox")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python")

In [2]:
import Bin_Packing_Py as BP
import BP_print
import BP_plot
import BP_utilities
import HA_routines
import BP_instances

import matplotlib
%matplotlib nbagg
matplotlib.rcParams['figure.figsize'] = [6,3.5]

from ortools.linear_solver import pywraplp
import random
import time
import csv

import pandas as pd
from sqlalchemy import create_engine
from mail import send_mail
from Test_BP_G import euristic

In [3]:
class model_bin_packing:
    def __init__(self,item_list,bin_list):
        self.bp = BP.BinPack()
        self.bp.items.extend(item_list)
        self.bp.bins.extend(bin_list)

    def buildModel(self,amount_time=30000):
        solver = pywraplp.Solver('Model3D', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        #importo dimensioni dei pacchi e del camion
        n=len(self.bp.items)

        w =[self.bp.items[i].width for i in range(n)]
        d =[self.bp.items[i].depth for i in range(n)]
        h =[self.bp.items[i].height for i in range(n)]
        
        W =self.bp.bins[0].width
        D =solver.NumVar(0,sum([max(w[i],d[i]) for i in range(n)]),"D")
        H =self.bp.bins[0].height

        Mw=W+max(max(w),max(d))
        Md=self.bp.bins[0].depth+max(max(w),max(d))
        Mh=self.bp.bins[0].height+max(h)
                
        # definisco le variabili
        l =[[solver.BoolVar("l%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        b =[[solver.BoolVar("b%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        t =[[solver.IntVar(0,self.bp.items[i].stackable,"t%d%d" % (i,j)) for j in range(n)] for i in range(n)]
        f =[[solver.IntVar(0,self.bp.items[i].stackable,"f%d%d" % (i,j)) for j in range(n)] for i in range(n)]
        k =[[solver.BoolVar("k%d%d" % (i,j)) for j in range(n)] for i in range(n)]
        r =[solver.BoolVar("r%d" % (i)) for i in range(n)]

        x =[solver.NumVar(0,solver.infinity(),"x%d" % i) for i in range(n)]
        y =[solver.NumVar(0,solver.infinity(),"y%d" % i) for i in range(n)]
        z =[solver.NumVar(0,solver.infinity(),"z%d" % i) for i in range(n)]
        
        #definisco i constraints
        for i in range(n):
            for j in range(n):
                if(i < j):
                    solver.Add(k[i][j] <= l[i][j] + l[j][i] + b[i][j] + b[j][i])
                    solver.Add(2 * k[i][j] >= l[i][j] + l[j][i] + b[i][j] + b[j][i])
                    solver.Add(1 - k[i][j] == t[i][j] + t[j][i])
                if(i != j):
                    solver.Add(x[i] - x[j] + Mw * l[i][j] <= Mw - w[i]*(1-r[i]) - d[i]*r[i])
                    solver.Add(y[i] - y[j] + Md * b[i][j] <= Md - d[i]*(1-r[i]) - w[i]*r[i])
                    
                    solver.Add(x[i] - x[j] + Mw * l[i][j] + w[i]*(1-r[i])/2 + d[i]*r[i]/2 >= 0)   
                    solver.Add(y[i] - y[j] + Md * b[i][j] + d[i]*(1-r[i])/2 + w[i]*r[i]/2 >= 0)
                    
                    solver.Add(z[i] - z[j] + Mh * t[i][j] <= Mh - h[i])
                    solver.Add(z[i] - z[j] - Mh * f[i][j] >= - Mh - h[i])
                    solver.Add(f[i][j] <= t[i][j])

            solver.Add(x[i] + w[i]*(1-r[i]) + d[i]*r[i] <= W)
            solver.Add(y[i] + d[i]*(1-r[i]) + w[i]*r[i] <= D)
            solver.Add(z[i] + h[i] <= H)

        for j in range(n):
            solver.Add(solver.Sum([f[i][j] for i in range(n) if i!=j]) * Mh >= z[j])
            
        #funzione obiettivo
        objective = solver.Objective()
        objective.SetCoefficient(D,1)
        objective.SetMinimization()

        #soluzione
        solver.SetTimeLimit(amount_time)
        tm=time.time()
        solver.Solve()
        tm =time.time()-tm

        s = BP.Solution(self.bp)
        for i in range(len(self.bp.items)):
            rt = False if r[i].solution_value() == 0 else True
            s.pushItem(self.bp.items[i],0,BP.Point(x[i].solution_value(),y[i].solution_value(),z[i].solution_value()),rt,False)
        return s,tm,w,d,h,n
    
    def BuildSolution(self,s):
        print s
    
    def plotSolution(self,s):
        BP_plot.plotSolution(s)
    
    def plot3Dsolution(self,s):
        BP_plot.plot3DSolution(s,0)

    def feasible(self):
        return self.bp.bins[0].height*self.bp.bins[0].width >= sum([self.bp.items[i].width*self.bp.items[i].height for i in range(len(self.bp.items))])

In [4]:
n_items = []
Avg_w = []
Avg_d = []
Avg_a = []
Time = []
eu_obj = []
mod_obj = []
Avg_wd = []
square_index = []
feasible = []

In [5]:
for i in range(10):
    if (i % 25 == 0): print i
    items,bins=BP_instances.tc_sfida()
    model = model_bin_packing(items,bins)
    s,t,w,d,h,n=model.buildModel()
    s.computeObj()
    mod_obj.append(s.obj)
    eu_obj.append(euristic(items,bins))
    Time.append(round(t,3))
    Avg_w.append(round(sum(w)/n,3))
    Avg_d.append(round(sum(d)/n,3))
    square_index.append((min(Avg_w[i],Avg_d[i])/max(Avg_w[i],Avg_d[i])*100))
    A = [w[i]*d[i] for i in range(n)]
    Avg_a.append(sum(A)/n)
    n_items.append(n)
    feasible.append(min([s.checkStable(i,0) for i in s.getAnchors(0)]))

0


In [6]:
dimensioni =pd.DataFrame({'n_items': n_items, 'Square_index': square_index, 'Avg width': Avg_w,'Avg depth': Avg_d,'Avg Area': Avg_a,'Eu obj': eu_obj,'Mod obj': mod_obj, 'Time': Time, 'Stable': feasible})

In [7]:
dimensioni.to_excel('result.xlsx', encoding='utf8')
dimensioni.to_latex('result.tex', encoding='utf8')
dimensioni.to_csv('result.csv', encoding='utf8')

In [8]:
dimensioni

,Avg Area,Avg depth,Avg width,Eu obj,Mod obj,Square_index,Stable,Time,n_items
0,7.723519,3.511,2.192,3010.233181,13.955707,62.432355,False,22.565,8
1,8.064311,3.488,2.305,6013.291777,3.988306,66.083716,True,25.239,10
2,8.367085,3.757,2.228,11.272300,7.491611,59.302635,False,0.036,3
3,7.920939,3.512,2.247,2010.741657,13.316703,63.980638,False,2.908,7
4,7.784203,3.530,2.210,1010.861801,10.862770,62.606232,False,8.794,6
5,7.522856,3.251,2.313,13.114774,13.112745,71.147339,False,0.875,5
6,7.930443,3.649,2.176,5011.342525,15.251586,59.632776,False,30.662,10
7,7.260873,3.505,2.072,2010.736128,14.012516,59.115549,False,30.532,8
8,8.187817,3.602,2.276,4011.171884,14.582921,63.187118,False,20.130,8
9,7.651522,3.333,2.289,1013.257184,12.965690,68.676868,False,12.078,8


In [9]:
send_mail()